# Tutorial: SOMA Objects

In this notebook, we'll go through the various objects available as part of the SOMA API. The dataset used is from Peripheral Blood Mononuclear Cells (PBMC), which is freely available from 10X Genomics. 

We'll start by importing `tiledbsoma`.

In [1]:
import tiledbsoma

## Experiment

An `Experiment` is a class that represents a single-cell experiment. It always contains two objects:
1. `obs`: A `DataFrame` with primary annotations on the observation axis.
2. `ms`: A `Collection` of measurements.

In [2]:
experiment = tiledbsoma.open("data/dense/pbmc3k")
experiment

Each object can be opened like this:

In [3]:
experiment.ms

In [4]:
experiment.obs

Note that by default an `Experiment` is opened lazily, i.e. only the minimal requested objects are opened. 

Also, opening an object doesn't mean that it will entirely be fetched in memory. It only returns a pointer to the object on disk.

## DataFrame

A `DataFrame` is a multi-column table with a user-defined schema. The schema is expressed as an Arrow Schema, and defines the column names and value types.

As an example, let's take a look at `obs`, which is represented as a SOMA DataFrame.

We can inspect the schema using `.schema`:

In [5]:
obs = experiment.obs
obs.schema

Note that `soma_joinid` is a field that exists in each `DataFrame` and acts as a join key for other objects, such as `SparseNDArray` (more on this later).

When a `DataFrame` is accessed, only metadata is retrieved, not actual data. This is important since a DataFrame can be very large and might not fit in memory.

To materialize the dataframe (or a subset) in memory, we call `df.read()`. 

If the dataframe is small, we can convert it to an in-memory Pandas object like this:

In [6]:
obs.read().concat().to_pandas()

Here, `read()` returns an iterator, `concat()` materializes all rows to memory and `to_pandas()` returns a Pandas view of the dataframe.

If the dataframe is bigger, we can only select a subset of it before materializing. This will only retrieve the required subset from disk to memory, so very large dataframes can be queried this way. In this example, we will only select the first 10 rows:

In [7]:
obs.read((slice(0,10),)).concat().to_pandas()

We can also select a subset of the columns:

In [8]:
obs.read((slice(0, 10),), column_names=["obs_id", "n_genes"]).concat().to_pandas()

Finally, we can use `value_filter` to retrieve a filtered subset of rows that match a certain condition.

In [9]:
obs.read((slice(None),), value_filter="n_genes > 1500").concat().to_pandas()

## Collection

A `Collection` is a persistent container of named SOMA objects, stored as a mapping of string keys and SOMA object values.

The `ms` member in an Experiment is implemented as a Collection. Let's take a look:

In [10]:
experiment.ms

In this case, we have two members: `raw` and `test_exp_name`. They can be accessed as they were dict members:

In [11]:
experiment.ms["raw"]

## DenseNDArray

A ``DenseNDArray`` is a dense, N-dimensional array, with offset (zero-based) integer indexing on each dimension. 

`DenseNDArray` has a user-defined schema, which includes:
- the element type, expressed as an Arrow type, indicating the type of data contained within the array, and
- the shape of the array, i.e., the number of dimensions and the length of each dimension

In a SOMA single cell experiment, the cell by gene matrix X is typically represented either by `DenseNDArray` or `SparseNDArray`. Let's take a look at our example:

In [12]:
X = experiment["ms"]["RNA"].X
X

Within the experiment, `X` is a `Collection` and the data can be accessed using `["data"]`:

In [13]:
X = X["data"]
X

We can inspect the `DenseNDArray` and get useful information by using `.schema`:

In [14]:
X.schema

In this case, we see there are two dimensions and the data is of type `float`.

We can see the shape of the matrix by calling `.shape`. In this case, since this represents a dense matrix, this will be the exact size of the matrix:

In [15]:
X.shape

Similarly to `DataFrame`, when opening a `DenseNDArray` only metadata is fetched, and the array isn't fetched into memory. 

We can convert the matrix into a `pyarrow.Tensor` using `.read()`:

In [16]:
X.read()

From here, we can convert it further to a `numpy.ndarray`:

In [17]:
X.read().to_numpy()

This will only work on small matrices, since a `numpy` array needs to be in memory. 

We can retrieve a subset of the matrix passing coordinates to `.read()`. Here we're only retrieving the first 10 rows of the matrix:

In [18]:
sliced_X = X.read((slice(0,9),)).to_numpy()
sliced_X

In [19]:
sliced_X.shape

Note that `DenseNDArray` is always indexed, on each dimension, using zero-based integers. If this dimension matches any other object in the experiment, the `soma_joinid` column can be used to retrieve the correct slice.

In the following example, we will get the values of X for the gene tagged as `ICOSLG`. This involves reading the `var` DataFrame using a `value_filter`, retrieving the `soma_joinid` for the gene and passing it as coordinate to `X.read`:


In [20]:
var = experiment.ms["RNA"].var
idx = var.read(value_filter="var_id == 'ICOSLG'").concat()["soma_joinid"].to_numpy()

X.read((None, int(idx[0]))).to_numpy()

## SparseNDArray

A `SparseNDArray` is a sparse, N-dimensional array, with offset (zero-based) integer indexing on each dimension. `SparseNDArray` has a user-defined schema, which includes:
- the element type, expressed as an Arrow type, indicating the type of data
      contained within the array, and
- the shape of the array, i.e., the number of dimensions and the length of
      each dimension

A `SparseNDArray` is functionally similar to a `DenseNDArray`, except that only elements that have a nonzero value are actually stored. Elements that are not explicitly stored are assumed to be zeros.

As an example, we will load a version of pbmc3k that has been generated using a `SparseNDArray`:

In [21]:
experiment = tiledbsoma.open("data/sparse/pbmc3k")
X = experiment.ms["RNA"].X["data"]
X

Let's take a look at the schema:

In [22]:
X.schema

This is the same as the `DenseNDArray` version, which makes sense since it's still a 2-dimensional matrix with `float` data.

Let's look at the shape:

In [23]:
X.shape

Since sparse matrices are not represented as contiguous arrays in memory, they don't have a fixed size like a dense matrix would have. Instead, `.shape()` returns the _capacity_ of the matrix, which means that those are valid indices for reading/writing to that matrix. These are dependent on the capacity of the system rather than the current bounding box of the array.

The closest concept to size for a `SparseNDArray` is the non-empty domain which can be defined as the largest coordinates that correspond to a nonzero value (across each dimension). There is currently no direct way to infer the nonzero domain of a `SparseNDArray` without materializing the array; however, `obs.count` and `var.count` provide these values.

We can get the number of nonzero elements by calling `.nnz`:

In [24]:
X.nnz

In order to work with a `SparseNDArray`, we call `.read()`:

In [25]:
X.read()

This returns a SparseNDArrayRead that can be used for getting iterators. For instance, we can do:

In [26]:
tensor = X.read().coos().concat()

This returns an [Arrow Tensor](https://arrow.apache.org/docs/cpp/api/tensor.html) that can be used to access the array, or convert it further to different formats. For instance:

In [27]:
tensor.to_scipy()

can be used to transform it to a [SciPy coo_matrix](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html). 

Similarly to `DenseNDArray`s, we can call `.read()` with a slice to only obtain a subset of the matrix. As an example:

In [28]:
sliced_X = X.read((slice(0,9),)).coos().concat().to_scipy()
sliced_X

Let's verify that the slice is correct. To do that, we can call `nonzero()` on the `scipy.sparse.coo_matrix` to obtain the coordinates of the nonzero items, and look at the coordinates over the first dimension:

In [29]:
sliced_X.nonzero()[0]